In [1]:
# Suppress TensorFlow warnings and info messages
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Available backend options are: "jax", "torch", "tensorflow".
os.environ["KERAS_BACKEND"] = "tensorflow"
# os.environ["KERAS_BACKEND"] = "jax"

import numpy as np
import keras
import json
import cv2

# Load a model from Hugging Face Hub
# The model is a FaceNet512 model trained on the VGGFace2 dataset.
# It can be used for face recognition tasks.
# The model is saved in the TensorFlow format.
# The custom object 'scaling' is used to scale the input images.
# The model can be used to extract face embeddings from images.
@keras.saving.register_keras_serializable()
def scaling(x, scale=1.0):
	return x * scale

model = keras.saving.load_model(
	"hf://logasja/FaceNet512",
	custom_objects={"scaling": scaling}
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [25]:
# Access camera
def setCamera(index, width=640, height=480, fps=30):
    cap = cv2.VideoCapture(index)
    
    # Set the camera resolution (optional)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    
    # Set the camera frame rate (optional)
    cap.set(cv2.CAP_PROP_FPS, fps)
    
    return cap

# Change the camera index if necessary
# Common indices are 0 for the default camera, 1 for an external camera, etc.
def getCamera(index=1, max_attempt=3):
    while max_attempt:
        print(f"Opening Camera {index}...")
        cap = setCamera(index)
        if not cap.isOpened():
            print(f"Camera {index} is not Available!")
            index+=1
            max_attempt-=1
        else:
            print(f"Camera {index} is Available!")
            return cap
    
    if not max_attempt:
        print("Opening Default Camera")
        cap = setCamera(index=0)
        if not cap.isOpened():
            print("Default Camera is not Available!")
            exit()
        return cap
    
# Detect faces in the camera feed
# This function uses OpenCV's Haar Cascade classifier to detect faces in the video frames.`
# It returns the coordinates of the detected faces.
def detectFace(frame):
    HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
    
    # Detect faces in the image
    faces = HaarCascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=5)
    if len(faces) > 0:
        print(f"Detected {len(faces)} face(s).")
        return faces
    else:
        print("No faces detected.")
        return None
    
# Render a rectangle around the detected faces
def drawRectangle(frame, faces):
    for (x, y, width, height) in faces:
        x, y = abs(x), abs(y)
        frame = cv2.rectangle(frame, (x,y), (x+width, y+height), (0,0,255), 2)
    return frame

# Pre Predict function to boostup the performance
def prePredict():
    # Preload the model to boostup the performance
    imgTest = np.zeros((1, 160, 160, 3), dtype=np.float32)
    signature = model.predict(imgTest)
    print("Model preloaded successfully.")
    return signature
    
# Extract face embeddings from the detected faces
def extractFaceEmbeddings(faces, frame):
    for (x, y, width, height) in faces:
        # Extract the face region from the frame
        face = frame[y:y+height, x:x+width]

        # Resize face to the input size expected
        face = cv2.resize(face, (160, 160))

        # Normalize the facce image
        face = face / 255.0

        # Add batch dimension
        face = np.expand_dims(face, axis=0)

        signature = model.predict(face)
    return signature[0]

# Save the face embeddings to a file
def saveFaceEmbeddings(signature, filename="signatures.json"):
    if os.path.exists(filename):
        print(f"File {filename} already exists. Updating the file.")
    else:
        print(f"File {filename} does not exist. Creating a new file.")
        json.dump({}, open(filename, mode="w"), indent=4)
    
    data = json.load(open(filename, mode="r"))
    
    data.update({
        "name": "Yasir",
        "signature": signature.tolist()
    })
    
    json.dump(data, open(filename, mode="w"), indent=4)
    
# Face recognition function
def faceRecognition(frame, embeddedFaces, filename="signatures.json"):
    faceDatabase = json.load(open(filename, mode="r"))
    
    # for key, value in faceDatabase.items():
    #     value = np.array(value)
    #     print(key, type(value))
        
    
    # Calculate the Euclidean distance between the embedded faces and the stored signatures
    distance = np.linalg.norm(faceDatabase['signature'] - embeddedFaces)
    print(f"Distance: {distance}")
    
    if (distance > 20):
        cv2.putText(frame, "Unknown, {}".format(distance), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)
    else:
        identity = faceDatabase['name']
        cv2.putText(frame, identity + ", " + str(round(distance, 2)), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)
                        

In [26]:
# Access camera
cap = getCamera()

# Check if the video capture has been initialized correctly
if not cap.isOpened():
    print("Error: Could not open video device.")
    exit()

# Preload the model to boostup the performance
prePredict()

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("Could not read frame.")
        break

    # Convert the frame to RGB format
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces in the frame
    # The detectFace function returns the coordinates of detected faces/*
    # If no faces are detected, it returns None
    faces = detectFace(frame)

    if faces is not None:
        # Draw rectangles around detected faces
        frame = drawRectangle(frame, faces)
        # Here you can add code to process the detected faces, e.g., extract embeddings
        # or perform recognition using a pre-trained model.
        embeddedFaces = extractFaceEmbeddings(faces, frame)
        # print(type(embeddedFaces))
        
        # Save the face embeddings to a file
        # saveFaceEmbeddings(embeddedFaces)
        
        # Face Recognition Processing
        faceRecognition(frame, embeddedFaces)

    # Convert the frame to BGR format
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    # Display the frame in a window
    cv2.imshow("Camera Feed", frame)

    # Wait for 1 ms and check if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
# Note: The video capture object is released and all OpenCV windows are closed
# when the loop is exited, either by pressing 'q' or if an error occurs.
# This ensures that resources are cleaned up properly.
cap.release()
cv2.destroyAllWindows()

Opening Camera 1...
Camera 1 is not Available!
Opening Camera 2...
Camera 2 is not Available!
Opening Camera 3...
Camera 3 is not Available!
Opening Default Camera


[ WARN:0@830.421] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@830.518] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@830.518] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@830.519] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@830.519] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video3): can't open camera by index
[ERROR:0@830.520] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Model preloaded successfully.
Detected 1 face(s).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Distance: 19.0696006476574
Detected 1 face(s).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Distance: 18.968106892856028
Detected 1 face(s).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Distance: 19.832116181272344
Detected 1 face(s).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Distance: 20.207722073148346
Detected 1 face(s).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Distance: 20.535419085017676
Detected 1 face(s).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Distance: 17.396417246363132
Detected 1 face(s).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Distance: 20.30978730130475
Detected 1 face(s).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Distance: 18.7442797770169
Detected 1 face(s).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Distance: 21.09338771524486
Detected 1 face(s).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Distance: 17.956203150610026
Detected 1 face(s).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Distance: 